In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
import timeit

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# data = []
# total_requests = 0

# # Convert initial date string to datetime object
# date = datetime.fromisoformat('2024-12-01T16:10:00.740000+00:00')

# start_time = timeit.default_timer()

# while True:
#     # Convert datetime back to string for URL
#     date_start = date.isoformat()
#     date_end = (date + timedelta(seconds=1)).isoformat()
#     url = f'https://api.openf1.org/v1/car_data?session_key=9655&date>={date_start}&date<={date_end}'
#     print(url)
#     try:
#         response = requests.get(url)
#         data = response.json()
#         # Convert to pandas DataFrame
#         df = pd.DataFrame(data)
#     except requests.RequestException as e:
#         print(f"Request error: {e}")
#         continue
#     except json.JSONDecodeError as e:
#         print(f"JSON decode error: {e}")
#         continue
    
#     if df.empty:
#         continue

#     # Increment datetime object
#     date += timedelta(seconds=1)
#     total_requests += 1
    
#     # Clear the output in Jupyter
#     clear_output(wait=True)
    
#     # Print stats and display DataFrame
#     elapsed_time = timeit.default_timer() - start_time
#     print(f"Elapsed time: {elapsed_time:.2f} seconds")
#     print(f"Seconds per loop: {elapsed_time / total_requests:.3f}")
#     print(f"Total requests: {total_requests}")
#     print("\nCurrent car data for all drivers:")
#     display(df[['driver_number', 'date', 'rpm', 'speed', 'n_gear', 'throttle', 'drs', 'brake']]
#            .sort_values(['date'], ascending=[False])
#            .groupby('driver_number')
#            .first()
#            .reset_index()
#            .sort_values('driver_number'))
#     print("\n")


In [ ]:
import plotly.express as px
import pandas as pd

drivers = [1, 63]
all_data = []

for driver in drivers:
    url = f'https://api.openf1.org/v1/car_data?driver_number={driver}&session_key=9655&date>={datetime.fromisoformat('2024-12-01T16:00:00.740000+00:00')}&date<={datetime.fromisoformat('2024-12-01T16:15:01.740000+00:00')}'
    response = requests.get(url)
    data = response.json()
    # Add driver number to each row for coloring
    for row in data:
        row['driver'] = str(driver)
    all_data.extend(data)

df = pd.DataFrame(all_data)

fig = px.line(df, x='date', y='speed', color='driver',
              title='Speed Over Time by Driver',
              labels={'speed': 'Speed (km/h)', 'date': 'Time'})
fig.show()

In [ ]:
import time
from IPython.display import HTML, clear_output

# First get track outline from historical data
url = f'https://api.openf1.org/v1/location?session_key=9655&driver_number=1&date>={datetime.fromisoformat('2024-12-01T16:00:00.740000+00:00')}&date<={datetime.fromisoformat('2024-12-01T16:15:01.740000+00:00')}'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
track_points = df[['x', 'y']].drop_duplicates().values.tolist()

# Scale track points to fit SVG viewport (800x600)
x_vals = [p[0] for p in track_points]
y_vals = [p[1] for p in track_points]
x_min, x_max = min(x_vals), max(x_vals)
y_min, y_max = min(y_vals), max(y_vals)

def scale_point(x, y):
    # Scale to fit 800x600 with 20px padding
    x_scaled = 20 + (x - x_min) * 760 / (x_max - x_min)
    y_scaled = 20 + (y - y_min) * 560 / (y_max - y_min)
    return x_scaled, y_scaled

# Scale track points
scaled_track = [scale_point(x, y) for x, y in track_points]

# Create SVG template with three dots
svg_template = '''
<svg width="800" height="600" style="background: black;">
    <path d="M {} Z" stroke="white" stroke-width="2" fill="none"/>
    <circle cx="{}" cy="{}" r="5" fill="red"/>
    <circle cx="{}" cy="{}" r="5" fill="blue"/>
    <circle cx="{}" cy="{}" r="5" fill="green"/>
</svg>
'''

# Create track path
track_path = ' '.join(f"{x},{y} L" for x, y in scaled_track)[:-2]  # Remove last "L"

# Initialize start time
start_time = datetime.fromisoformat('2024-12-01T16:00:10.740000+00:00')
end_time = datetime.fromisoformat('2024-12-01T16:15:01.740000+00:00')

drivers = [1, 4, 10, 11, 14, 16, 18, 20, 22, 23, 24, 27, 31, 44, 55, 63, 77, 81]

while start_time < end_time:
    try:
        all_positions = []
        
        # Get position between start_time and start_time + 1s for all drivers
        for driver in drivers:
            url = f'https://api.openf1.org/v1/location?session_key=9655&driver_number={driver}&date>={start_time}&date<={start_time + timedelta(seconds=1)}'
            response = requests.get(url)
            data = response.json()
            if data:
                all_positions.extend(data)
        
        # Check if we got any data
        if not all_positions:
            print("No data received, moving forward...")
            start_time += timedelta(seconds=1)
            continue
            
        current_pos_df = pd.DataFrame(all_positions)
        if len(current_pos_df) == 0:
            print("Empty dataframe, moving forward...")
            start_time += timedelta(seconds=1)
            continue
            
        # Get latest position for each driver
        latest_positions = current_pos_df.groupby('driver_number').last().reset_index()
        
        # Scale positions for all three drivers
        scaled_positions = []
        for _, pos in latest_positions.iterrows():
            driver_x, driver_y = scale_point(pos['x'], pos['y'])
            scaled_positions.extend([driver_x, driver_y])
        
        # Fill in missing drivers with off-screen coordinates
        while len(scaled_positions) < 6:  # Need 6 values for 3 drivers
            scaled_positions.extend([-10, -10])  # Off-screen coordinates
            
        # Create SVG with current positions
        svg = svg_template.format(track_path, *scaled_positions)
        
        # Clear output and display
        clear_output(wait=True)
        display(HTML(svg))
        
        # Add small delay to avoid hammering the API
        time.sleep(0.1)
        
        # Move forward in time
        start_time += timedelta(seconds=1)
        
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(0.1)
        start_time += timedelta(seconds=1)
%%timeit
a = 1

In [12]:


import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta
import os

# print("Starting database setup...")

# Delete existing database if it exists
if os.path.exists('racing_data.db'):
    os.remove('racing_data.db')
    # print("Deleted existing database")

# Create connection to SQLite database
conn = sqlite3.connect('racing_data.db')
cursor = conn.cursor()

# print("Connected to database successfully")

# Define 3 core tables
table_configs = [
    {
        'name': 'drivers',
        'columns': ['driver_id', 'name', 'nationality', 'team', 'age', 'experience']
    },
    {
        'name': 'lap_times', 
        'columns': ['race_id', 'driver_id', 'lap_number', 'lap_time', 'position', 'timestamp']
    },
    {
        'name': 'telemetry',
        'columns': ['race_id', 'driver_id', 'speed', 'gear', 'throttle', 'timestamp']
    }
]

# Create tables and insert random data
for config in table_configs:
    # print(f"Creating table {config['name']}...")
    
    # Create table
    columns_sql = ', '.join([f"{col} TEXT" for col in config['columns']])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {config['name']} ({columns_sql})")
    
    # print(f"Generating random data for {config['name']}...")
    
    # Generate 1000 rows of random data
    data = []
    rows_to_insert = 60000
    for i in range(rows_to_insert):
        row = [str(random.randint(1, 100)) for _ in range(len(config['columns']))]
        data.append(row)
        # if (i + 1) % rows_to_insert / 10 == 0:
            # print(f"Generated {i + 1} rows...")
    
    # print(f"Inserting data into {config['name']}...")
    
    # Insert data
    placeholders = ','.join(['?' for _ in config['columns']])
    cursor.executemany(f"INSERT INTO {config['name']} VALUES ({placeholders})", data)
    # print(f"Finished inserting data into {config['name']}")

conn.commit()
# print("All data committed successfully")

# Complex query with joins and aggregations
# print("Running test query...")
query = """
SELECT 
    d.name
    , d.team
    , sum(nationality)
FROM drivers d
group by 1,2
"""

result_df = pd.read_sql_query(query, conn)
# print("\nQuery results:")
# print(result_df)

conn.close()
# print("\nDatabase connection closed")
print('done')

done


In [ ]:
%%timeit
a = 1

In [1]:
from db_utils import QuestDBClient

def main():
    db = QuestDBClient()
    
    # Insert example
    data = {
        'symbol': 'BTC-USD',
        'price': 50000,
        'amount': 1.5
    }
    db.insert_data('trades', data)
    
    # Query example
    query = "SELECT * FROM trades ORDER BY timestamp DESC LIMIT 10"
    results = db.query_data(query)
    print(results)


main()

Insert error: missing trailing semicolon at position 25
Query error: (psycopg2.DatabaseError) unknown function name: EXISTS(CURSOR)
LINE 1: SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE...
               ^

[SQL: SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'trades');]
(Background on this error at: https://sqlalche.me/e/20/4xp6)
Empty DataFrame
Columns: []
Index: []


In [14]:
import os

def list_files_in_subdirs():
    # Get current directory
    current_dir = os.getcwd()
    
    # Store files found in subdirectories
    files_in_subdirs = []
    
    # Walk through directory tree
    for root, dirs, files in os.walk(current_dir):
        # Skip the current directory
        if root == current_dir:
            continue
            
        # Add files with their relative paths
        for file in files:
            rel_path = os.path.relpath(os.path.join(root, file), current_dir)
            files_in_subdirs.append(rel_path)
            
    return files_in_subdirs

# Print all files found in subdirectories
files = list_files_in_subdirs()
for file in files:
    if '.git' not in file:
        print(file)



real-time-f1/pnpm-lock.yaml
real-time-f1/postcss.config.mjs
real-time-f1/next.config.mjs
real-time-f1/next-env.d.ts
real-time-f1/README.md
real-time-f1/tailwind.config.ts
real-time-f1/package.json
real-time-f1/components.json
real-time-f1/tsconfig.json
real-time-f1/.eslintrc.json
real-time-f1/app/favicon.ico
real-time-f1/app/layout.tsx
real-time-f1/app/page.tsx
real-time-f1/app/globals.css
real-time-f1/app/api/route.ts
real-time-f1/app/fonts/GeistMonoVF.woff
real-time-f1/app/fonts/GeistVF.woff
real-time-f1/node_modules/.modules.yaml
real-time-f1/node_modules/.pnpm/lock.yaml
real-time-f1/node_modules/.pnpm/is-extglob@2.1.1/node_modules/is-extglob/LICENSE
real-time-f1/node_modules/.pnpm/is-extglob@2.1.1/node_modules/is-extglob/index.js
real-time-f1/node_modules/.pnpm/is-extglob@2.1.1/node_modules/is-extglob/README.md
real-time-f1/node_modules/.pnpm/is-extglob@2.1.1/node_modules/is-extglob/package.json
real-time-f1/node_modules/.pnpm/object.assign@4.1.5/node_modules/object.assign/LICENSE


In [4]:
import time
import numpy as np

def test_sleep_accuracy(sleep_times):
    results = {}
    
    for sleep_time in sleep_times:
        deltas = []
        
        # Test each sleep duration 100 times
        for _ in range(1000):
            start = time.perf_counter()
            time.sleep(sleep_time)
            actual = time.perf_counter() - start
            deltas.append(actual - sleep_time)
            
        results[sleep_time] = {
            'mean_error': np.mean(deltas),
            'std_error': np.std(deltas),
            'min_error': min(deltas),
            'max_error': max(deltas)
        }
        
    return results

# Test different sleep durations
sleep_times = [0.1]
results = test_sleep_accuracy(sleep_times)

# Print results
for sleep_time, metrics in results.items():
    print(f"\nResults for sleep duration {sleep_time}s:")
    print(f"Mean error: {metrics['mean_error']*1000:.2f}ms")
    print(f"Std dev error: {metrics['std_error']*1000:.2f}ms") 
    print(f"Min error: {metrics['min_error']*1000:.2f}ms")
    print(f"Max error: {metrics['max_error']*1000:.2f}ms")



Results for sleep duration 0.1s:
Mean error: 3.57ms
Std dev error: 1.51ms
Min error: 0.05ms
Max error: 10.00ms


In [11]:
%%timeit -n 100
time.sleep(0.1)

KeyboardInterrupt: 